In [ ]:
import os
import time
import requests
from urllib.parse import urljoin
from dotenv import load_dotenv
from bs4 import BeautifulSoup

load_dotenv()
BASE_URL = os.getenv("url")

# the three counties of concern
counties = ["Douglas", "Lancaster", "Sarpy"]
# years to scrape
years = [str(x) for x in range(2025, 2026)]

# where to dump the HTML
OUTPUT_DIR = "./CaseSearchAll"
os.makedirs(OUTPUT_DIR, exist_ok=True)

session = requests.Session()

county = "Lancaster"
year = 2025

# 1) GET the search page once to pick up cookies & hidden form values
r = session.get(BASE_URL)
r.raise_for_status()
soup = BeautifulSoup(r.text, "html.parser")

# 2) Gather all hidden inputs so we don’t break 
form = soup.find("form", id="searchForm")  # adjust form id/name as needed
payload = {}
for inp in form.find_all("input", {"type": ["hidden", "text"]}):
    name = inp.get("name")
    if not name:
        continue
    payload[name] = inp.get("value", "")

# 3) Override the dropdowns/radios with our values
#    You must confirm these field names ("county_num", "court_type", etc.)
payload.update({
    "county_num": county,
    "court_type": "All Court Types",
    "case_type": "Juvenile",
    "judge": "All Judges",
    "attorney_name": "All Attorneys",
    "year": year,
    "sort": "casenum",
    "order": "desc",     # assuming the “Descending” radio sets order=desc
    # plus the name/value of your submit button if needed, e.g.:
    # "search_button": "Search"
})

# 4) POST the filled‐in form
r = session.post(BASE_URL, data=payload)
r.raise_for_status()

page_number = 0

"""
while True:
    # save current page
    fname = f"{county}_{year}_{page_number}.html"
    with open(os.path.join(OUTPUT_DIR, fname), "w", encoding="utf-8") as f:
        f.write(r.text)
    print(f"Saved {fname}")

    # parse again to look for a “Next” link
    soup = BeautifulSoup(r.text, "html.parser")
    # adjust selector to match your “Next” link exactly
    next_a = soup.select_one("div#page_links.text-center li a:nth-of-type(2)")
    if not next_a or "Next" not in next_a.text:
        break

    # follow the Next href
    next_href = next_a["href"]
    next_url = urljoin(BASE_URL, next_href)
    time.sleep(1)           # be kind to the server
    r = session.get(next_url)
    r.raise_for_status()
    page_number += 1

# small pause before next county/year
time.sleep(2)
"""

AttributeError: 'NoneType' object has no attribute 'find_all'